In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
os.getcwd()

'/Users/miek/Documents/DS4B/DS4BThesisSNCB'

In [40]:
# read in data
weather_raw = pd.read_csv("~/Downloads/weather_data.csv")

# select only data from reference weather station
weather_raw = weather_raw[weather_raw['weather_station_id'] == 6447] # Ukkel/Uccle reference weather station
weather_raw = weather_raw.reset_index()

weather_raw['weather_date'] = pd.to_datetime(weather_raw['weather_date'])
# Extract the date and hour components into separate columns
weather_raw['weather_hour'] = weather_raw['weather_date'].dt.hour
weather_raw['weather_date'] = weather_raw['weather_date'].dt.date

weather_raw.iloc[:, :10].head()

,index,weather_station_id,weather_station_name,gps_latitude,gps_longitude,weather_date,precipitation_quantity,precipitation_range,temperature,minimum_temperature
0,6,6447,UCCLE-UKKEL,4.357871,50.796862,2021-10-21,NaN,NaN,5.7,NaN
1,58,6447,UCCLE-UKKEL,4.357871,50.796862,2020-02-27,9.4,2.0,2.5,NaN
2,97,6447,UCCLE-UKKEL,4.357871,50.796862,2021-10-21,NaN,NaN,10.5,NaN
3,108,6447,UCCLE-UKKEL,4.357871,50.796862,2022-04-04,NaN,NaN,5.3,NaN
4,165,6447,UCCLE-UKKEL,4.357871,50.796862,2021-03-13,NaN,NaN,6.4,NaN


In [41]:

# Filter the dataframe to include only rows between 6 AM and 9 AM
morning_data = weather_raw[(weather_raw['weather_hour'] >= 6) & (weather_raw['weather_hour'] < 10)].copy()

# Filter the dataframe to include only rows between 4 PM and 7 PM
evening_data = weather_raw[(weather_raw['weather_hour'] >= 16) & (weather_raw['weather_hour'] < 20)].copy()

# Set NaN values to zero for the specified columns
columns_to_fillna = ['precipitation_quantity', 'temperature', 'wind_speed', 'relative_humidity']
morning_data[columns_to_fillna] = morning_data[columns_to_fillna].fillna(0)
evening_data[columns_to_fillna] = evening_data[columns_to_fillna].fillna(0)

# Group morning data by date and calculate the average values for the specified columns
morning_data = morning_data.groupby('weather_date')[columns_to_fillna].mean().reset_index()

# Group evening data by date and calculate the average values for the specified columns
evening_data = evening_data.groupby('weather_date')[columns_to_fillna].mean().reset_index()

# Add a new column to morning_data and set its values to 1
morning_data = morning_data.assign(morning_rush=1)

# Add a new column to evening_data and set its values to 0
evening_data = evening_data.assign(morning_rush=0)

# Concatenate morning_data and evening_data into a single dataframe
weather_data = pd.concat([morning_data, evening_data], ignore_index=True)

weather_data

,weather_date,precipitation_quantity,temperature,wind_speed,relative_humidity,morning_rush
0,2019-01-01,0.025,6.625,3.14750,87.800,1
1,2019-01-02,0.050,4.700,2.96625,73.000,1
2,2019-01-03,0.000,3.900,2.10225,86.100,1
3,2019-01-04,0.050,3.200,1.57975,97.850,1
4,2019-01-05,0.250,5.500,3.78250,100.000,1
...,...,...,...,...,...,...
3233,2023-06-03,0.000,21.525,4.75500,29.525,0
3234,2023-06-04,0.000,22.725,3.89600,30.575,0
3235,2023-06-05,0.000,21.725,4.54600,47.625,0
3236,2023-06-06,0.000,23.500,3.70175,40.525,0


In [42]:
# add precipitation dummy
weather_data['precipitation_quantity'] = np.where(weather_data['precipitation_quantity'] > 0, 1, 0)

# rename precipitation_quantity to precipitation
weather_data = weather_data.rename(columns={'precipitation_quantity': 'precipitation'})

weather_data

,weather_date,precipitation,temperature,wind_speed,relative_humidity,morning_rush
0,2019-01-01,1,6.625,3.14750,87.800,1
1,2019-01-02,1,4.700,2.96625,73.000,1
2,2019-01-03,0,3.900,2.10225,86.100,1
3,2019-01-04,1,3.200,1.57975,97.850,1
4,2019-01-05,1,5.500,3.78250,100.000,1
...,...,...,...,...,...,...
3233,2023-06-03,0,21.525,4.75500,29.525,0
3234,2023-06-04,0,22.725,3.89600,30.575,0
3235,2023-06-05,0,21.725,4.54600,47.625,0
3236,2023-06-06,0,23.500,3.70175,40.525,0


In [43]:
weather_data.to_csv('~/Downloads/weather_data_clean.csv', index=False)